In [ ]:
from misc import load, load_class_labels
from data_loader import data_loader;
from tensorflow import keras;

dataset_root = "./dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train.csv', 1, sample_rate)
val_info = load(dataset_root + 'labels/validation.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

dl = data_loader(train_info, train_dir, 400, (224, 224));
dl_val = data_loader(val_info, train_dir, 400, (224, 224));


## Classification using handcrafted features

In [ ]:
from multiprocessing.pool import ThreadPool
from handcrafted import extract_features, color_histograms, sift, gabor_response, lbp
from tqdm import tqdm;
import numpy as np;

sampling = train_info.sample(frac=0.5)
images_names = sampling.iloc[:, 0].values

pool = ThreadPool(processes=14)

angles = np.arange(0, np.pi, np.pi/4)
lambdas = np.arange(0, 1, 0.2);
gammas = [0.5]
max_features = 50

sift_features = pool.apply_async(extract_features, (train_dir, images_names,
                                                  [(lambda img: sift(img, max_features=max_features))]))

gabor = pool.apply_async(extract_features, (train_dir, images_names,
                                                  [(lambda img: gabor_response(img, (10, 10), angles, [5], lambdas, gammas))]))

color = pool.apply_async(extract_features, (train_dir, images_names,
                                                  [(lambda img: color_histograms(img))]))

lbp_features = pool.apply_async(extract_features, (train_dir, images_names,
                                                  [(lambda img: lbp(img))]))



 

sift_features = sift_features.get()
gabor = gabor.get()
color = color.get()
lbp_features = lbp_features.get()

In [ ]:
from misc import unroll_arrays
import bag_of_words as bow

prob = 1
voc_size = 300

unrolled = unroll_arrays(sift_features, 1);

kmeans = bow.fit(unrolled, vocabulary_size=voc_size, verbose=True, n_init=1, max_iter=500)

In [ ]:
bow_features = bow.predict(kmeans, sift_features)
print(bow_features.shape)

In [ ]:
unrolled_color = np.array(color)
unrolled_color = unrolled_color.reshape(unrolled_color.shape[0], -1)
unrolled_gabor = np.array(gabor)
unrolled_lbp = np.array(lbp_features)

all_features = np.concatenate((unrolled_color, bow_features, unrolled_gabor, unrolled_lbp), axis=1)
print(all_features.shape)

## Standard classification

### SVM

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(all_features, sampling['label'])

## Neural network classification

In [ ]:
model = keras.Sequential([
    keras.Input(shape=390),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(251, activation='softmax')
])

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)

In [ ]:
import pandas as pd
epoch_num = 40;

labels = keras.utils.to_categorical(sampling['label'].values.tolist(), num_classes=251)

for epoch in range(epoch_num):
    #df = df.sample(frac=1);
    model.fit(all_features, labels, batch_size=100, shuffle=True, validation_split=0.1);

In [ ]:
'''
for epoch in range(epoch_num):
    dl.shuffle_dataframe();
    for batch_num in tqdm(range(dl.number_of_batch())):
        (images, labels, _) = dl.get_batch(batch_num)
        features = []
        for image in images:
            image = np.float32(image)
            hist = color_histograms(image);
            features.append(hist)
        
        features = bow.predict(kmeans, features);
        
        labels = keras.utils.to_categorical(labels, num_classes=251)
        model.train_on_batch(features, labels)
'''

In [ ]:
dl_val.shuffle_dataframe()
for batch_num in tqdm(range(dl_val.number_of_batch())):
    (images, labels, _) = dl_val.get_batch(batch_num)
    features = []
    for image in images:
        image = cv.normalize(image, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        image = image[:, :, ::-1];
        #sift_info = sift(image, max_features=max_features)
        features.append(sift_info)
    
    features = bow.predict(kmeans, features);
    
    labels = keras.utils.to_categorical(labels, num_classes=251)
    loss, acc = model.evaluate(features, labels);